In [35]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from yellowbrick.cluster import KElbowVisualizer
import random
import math

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'hhttps://docs.google.com/spreadsheets/d/1baKBXDs_PiIIYyUFmq0UOCBcMwq6DWji/edit?usp=sharing&ouid=100934670427194582945&rtpof=true&sd=true'
id = '1baKBXDs_PiIIYyUFmq0UOCBcMwq6DWji'
print (id) 

1baKBXDs_PiIIYyUFmq0UOCBcMwq6DWji


In [3]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('sales_df_completed_uc.xlsx')  
st_df1 = pd.read_excel('sales_df_completed_uc.xlsx')

In [4]:
len(st_df1)

30556

In [5]:
st_df1

,cust_id,status,category,payment_method,E Mail,respond_to_discount,total_spent,total_discount_received,no_of_transactions,days_till_last_interaction,...,age_binned_Middle_Age,age_binned_Old_age,Gender_F,Gender_M,Region_Midwest,Region_Northeast,Region_South,Region_West,cluster,cluster_Cat
0,4,complete,Mobiles & Tablets,easypay_voucher,reggie.doughty@gmail.com,1,27394.190,3406.010,41,93,...,1,0,0,1,1,0,0,0,3,2
1,15,complete,Home & Living,cod,debbie.diebold@bp.com,0,216.800,0.000,6,323,...,0,1,1,0,1,0,0,0,0,0
2,20,complete,Health & Sports,cod,eddy.pulver@yahoo.co.uk,0,28719.018,2875.682,11,93,...,0,0,0,1,0,0,1,0,3,5
3,21,complete,Beauty & Grooming,cod,adam.kan@shell.com,0,105.000,0.000,1,331,...,1,0,0,1,0,0,1,0,0,5
4,28,complete,Soghaat,cod,reinaldo.drain@yahoo.com,1,3971.300,500.000,11,224,...,0,1,0,1,0,0,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30551,113303,complete,Entertainment,cod,carrol.bishop@hotmail.com,0,1158.700,0.000,15,121,...,0,1,1,0,0,0,1,0,0,5
30552,113392,complete,Men's Fashion,cod,emmaline.johnson@gmail.com,0,191.700,0.000,3,129,...,1,0,1,0,0,0,0,1,0,0
30553,113565,complete,Mobiles & Tablets,cod,lauren.crimmins@ntlworld.com,0,180.000,0.000,3,124,...,0,1,0,1,0,0,1,0,0,5
30554,113675,complete,Women's Fashion,cod,dominic.vester@yahoo.com,0,354.500,0.000,10,117,...,0,1,0,1,0,0,1,0,0,7


In [6]:
st_df1.groupby('respond_to_discount').size()

respond_to_discount
0    21035
1     9521
dtype: int64

In [7]:
st_df2= st_df1[st_df1['respond_to_discount'] == 1]
st_df2

,cust_id,status,category,payment_method,E Mail,respond_to_discount,total_spent,total_discount_received,no_of_transactions,days_till_last_interaction,...,age_binned_Middle_Age,age_binned_Old_age,Gender_F,Gender_M,Region_Midwest,Region_Northeast,Region_South,Region_West,cluster,cluster_Cat
0,4,complete,Mobiles & Tablets,easypay_voucher,reggie.doughty@gmail.com,1,27394.19000,3406.01000,41,93,...,1,0,0,1,1,0,0,0,3,2
4,28,complete,Soghaat,cod,reinaldo.drain@yahoo.com,1,3971.30000,500.00000,11,224,...,0,1,0,1,0,0,0,1,0,5
8,44,complete,Mobiles & Tablets,Payaxis,kendall.ro@gmail.com,1,10711.07900,1548.42100,9,176,...,1,0,1,0,0,0,1,0,3,5
9,47,complete,Entertainment,Easypay_MA,jessie.schock@bp.com,1,18548.33000,4956.97000,32,158,...,1,0,0,1,0,1,0,0,3,0
14,71,complete,Appliances,Easypay,deetta.tucci@exxonmobil.com,1,10151.17601,1880.82399,56,106,...,0,1,1,0,1,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30489,106961,complete,Mobiles & Tablets,jazzvoucher,lakita.motes@gmail.com,1,5852.39200,678.80800,4,206,...,1,0,1,0,0,1,0,0,0,5
30491,106964,complete,Mobiles & Tablets,jazzvoucher,samira.lanphear@yahoo.com,1,4389.29400,509.10600,3,206,...,0,0,1,0,1,0,0,0,0,5
30497,106992,complete,Superstore,jazzvoucher,joe.guillemette@hotmail.com,1,746.41400,126.58600,4,206,...,0,0,0,1,0,0,0,1,0,5
30509,107423,complete,Men's Fashion,customercredit,karole.goo@hotmail.com,1,9627.53500,90.99000,10,136,...,0,0,1,0,0,0,0,1,3,0


**Systematic Sampling**

Using https://www.calculator.net/sample-size-calculator.html?type=1&cl=95&ci=5&pp=50&ps=30556&x=67&y=14 

In [8]:
population_total_discount_received_mean = round(st_df2['total_discount_received'].mean(),3)
population_total_discount_received_mean

1459.16

**For 90% C.I : Sample size : 264 , Interval : 9521/264=36**

In [9]:
#Define sample size
sample_size_ci90 = 264

In [10]:
def systematic_sampling(st_df2, step):
    np.random.seed(42)
    start = np.random.randint(0, step)
    indexes = np.arange(start,len(st_df2),step=step)
    
    systematic_sample = st_df2.iloc[indexes]
    return systematic_sample
    
# Obtain a systematic sample and save it in a new variable
systematic_sample_1 = systematic_sampling(st_df2, 36)
systematic_sample_1


,cust_id,status,category,payment_method,E Mail,respond_to_discount,total_spent,total_discount_received,no_of_transactions,days_till_last_interaction,...,age_binned_Middle_Age,age_binned_Old_age,Gender_F,Gender_M,Region_Midwest,Region_Northeast,Region_South,Region_West,cluster,cluster_Cat
80,533,complete,Appliances,easypay_voucher,augustus.look@exxonmobil.com,1,1505.970,327.930,3,376,...,1,0,0,1,0,0,0,1,0,5
220,2066,complete,Mobiles & Tablets,Easypay,ali.solberg@gmail.com,1,5014.300,1261.200,4,370,...,0,1,0,1,0,0,1,0,0,5
374,3816,complete,Mobiles & Tablets,Easypay,karrie.haslett@aol.com,1,15337.940,2284.860,34,180,...,0,1,1,0,0,1,0,0,3,2
536,5805,complete,Appliances,Payaxis,tommy.shryock@yahoo.com,1,3424.870,602.630,3,274,...,0,1,0,1,1,0,0,0,0,5
659,7194,complete,Men's Fashion,cod,tyron.mosely@ibm.com,1,5299.900,525.000,4,103,...,1,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28509,97889,complete,Appliances,Easypay,thomasine.brook@yahoo.com,1,3703.860,880.740,9,96,...,0,1,1,0,0,0,1,0,0,5
29470,104777,complete,Beauty & Grooming,cod,joseph.holm@sbcglobal.net,1,281.100,50.000,1,227,...,0,1,0,1,0,1,0,0,0,5
29528,104872,complete,Mobiles & Tablets,cod,nida.haydel@gmail.com,1,1686.000,400.000,2,227,...,0,1,1,0,0,0,0,1,0,5
29994,105742,complete,Appliances,cod,jon.mariscal@btinternet.com,1,6143.200,2100.000,3,211,...,0,0,0,1,0,0,0,1,0,5


**For 95% C.I : Sample size : 370 , Interval : 9521/370=25**

In [13]:
# Define sample size
sample_size_ci95 = 370

In [44]:
def systematic_sampling(st_df2, step):
    
    np.random.seed(42)
    start = np.random.randint(0, step)
    indexes = np.arange(start,len(st_df2),step=step)
    
    systematic_sample = st_df2.iloc[indexes]
    return systematic_sample
    
# Obtain a systematic sample and save it in a new variable
systematic_sample_2 = systematic_sampling(st_df2, 26)
systematic_sample_2

,cust_id,status,category,payment_method,E Mail,respond_to_discount,total_spent,total_discount_received,no_of_transactions,days_till_last_interaction,...,age_binned_Middle_Age,age_binned_Old_age,Gender_F,Gender_M,Region_Midwest,Region_Northeast,Region_South,Region_West,cluster,cluster_Cat
17,85,complete,Men's Fashion,cod,lissette.tims@earthlink.net,1,5636.500,1240.000,6,419,...,1,0,1,0,0,1,0,0,0,0
90,582,complete,Computing,easypay_voucher,kerry.farnum@hotmail.com,1,462.500,90.000,1,376,...,0,1,0,1,1,0,0,0,0,5
191,1644,complete,Appliances,easypay_voucher,mose.prevatte@gmail.com,1,995.940,130.660,4,372,...,1,0,0,1,0,1,0,0,0,5
292,2626,complete,Appliances,Easypay,glory.drewes@comcast.net,1,31798.020,1740.280,19,195,...,0,0,1,0,0,0,0,1,3,5
421,4499,complete,Mobiles & Tablets,Easypay,karri.tuma@aol.com,1,4203.620,631.080,14,124,...,1,0,1,0,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29476,104784,complete,Beauty & Grooming,Easypay_MA,tammy.pumphrey@yahoo.co.in,1,373.100,70.000,2,217,...,1,0,1,0,0,0,1,0,0,5
29515,104854,complete,Mobiles & Tablets,cod,albert.ankney@gmail.com,1,269.800,50.000,2,196,...,1,0,0,1,0,0,1,0,0,5
29569,104939,complete,Mobiles & Tablets,cod,ezra.carolina@btinternet.com,1,2702.216,380.384,5,206,...,1,0,0,1,0,0,0,1,0,5
30292,106342,complete,Mobiles & Tablets,jazzvoucher,samuel.mcelyea@yahoo.com,1,5852.392,678.808,4,208,...,0,0,1,0,0,0,1,0,0,2


**For 99% C.I : Sample size : 621 , Interval : 9521/621=15**

In [12]:
# Define sample size
sample_size_ci99 = 621

In [43]:
def systematic_sampling(st_df2, step):
    
    np.random.seed(42)
    start = np.random.randint(0, step)
    indexes = np.arange(start,len(st_df2),step=step)
    
    systematic_sample = st_df2.iloc[indexes]
    return systematic_sample
    
# Obtain a systematic sample and save it in a new variable
systematic_sample_3 = systematic_sampling(st_df2, 15)
systematic_sample_3

,cust_id,status,category,payment_method,E Mail,respond_to_discount,total_spent,total_discount_received,no_of_transactions,days_till_last_interaction,...,age_binned_Middle_Age,age_binned_Old_age,Gender_F,Gender_M,Region_Midwest,Region_Northeast,Region_South,Region_West,cluster,cluster_Cat
17,85,complete,Men's Fashion,cod,lissette.tims@earthlink.net,1,5636.500,1240.000,6,419,...,1,0,1,0,0,1,0,0,0,0
62,380,complete,Appliances,easypay_voucher,jimmy.cardwell@gmail.com,1,1977.920,95.580,12,227,...,0,0,0,1,0,0,1,0,0,5
101,719,complete,Mobiles & Tablets,Easypay,gregg.sheffield@hotmail.com,1,5865.298,675.602,9,198,...,0,0,0,1,1,0,0,0,0,5
161,1401,complete,Mobiles & Tablets,easypay_voucher,eloy.lumsden@aol.com,1,5275.440,453.960,3,369,...,1,0,0,1,0,0,0,1,0,5
224,2090,complete,Superstore,cod,torie.rainville@btinternet.com,1,5695.600,200.000,5,221,...,1,0,1,0,0,0,1,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29683,105125,complete,Men's Fashion,cod,nicky.kimball@gmail.com,1,119.400,180.000,1,225,...,0,1,0,1,0,0,0,1,0,0
30294,106347,complete,Mobiles & Tablets,jazzvoucher,caleb.westling@aol.com,1,3812.718,427.082,4,208,...,1,0,0,1,1,0,0,0,0,5
30312,106387,complete,Mobiles & Tablets,jazzvoucher,ebony.brochu@outlook.com,1,2582.216,355.384,4,208,...,1,0,1,0,1,0,0,0,0,5
30402,106657,complete,Superstore,cod,jerrell.pentecost@gmail.com,1,864.900,80.000,13,137,...,0,1,0,1,1,0,0,0,0,5


**Calculation Sample mean, sample standard deviation and standard error for each sample considering total_disocunt_recieved**

**Sample 1**

In [45]:
td_mean_s1 = np.mean(systematic_sample_1["total_discount_received"])

In [46]:
td_std_s1 = np.std(systematic_sample_1["total_discount_received"],ddof=1)

In [47]:
td_se_s1 = td_std_s1 / np.sqrt(sample_size_ci90)

In [48]:
print(f"Mean for sample 1: {td_mean_s1:.4f}")
print(f"Standard deviation for sample 1: {td_std_s1:.4f}")
print(f"Standard Error for sample 1: {td_se_s1:.4f}")

Mean for sample 1: 1306.9267
Standard deviation for sample 1: 1822.4166
Standard Error for sample 1: 112.1620


**Sample 2**

In [49]:
td_mean_s2 = np.mean(systematic_sample_2["total_discount_received"])

In [50]:
td_std_s2 = np.std(systematic_sample_2["total_discount_received"],ddof=1)

In [51]:
td_se_s2 = td_std_s2 / np.sqrt(sample_size_ci95)

In [52]:
print(f"Mean for sample 2: {td_mean_s2:.4f}")
print(f"Standard deviation for sample 2: {td_std_s2:.4f}")
print(f"Standard Error for sample 2: {td_se_s2:.4f}")

Mean for sample 2: 1631.3326
Standard deviation for sample 2: 2478.0369
Standard Error for sample 2: 128.8270


**Sample 3**

In [53]:
td_mean_s3 = np.mean(systematic_sample_3["total_discount_received"])

In [54]:
td_std_s3 = np.std(systematic_sample_3["total_discount_received"],ddof=1)

In [55]:
td_se_s3 = td_std_s3 / np.sqrt(sample_size_ci99)

In [56]:
print(f"Mean for sample 3: {td_mean_s3:.4f}")
print(f"Standard deviation for sample 3: {td_std_s3:.4f}")
print(f"Standard Error for sample 3: {td_se_s3:.4f}")

Mean for sample 3: 1421.9489
Standard deviation for sample 3: 2386.4088
Standard Error for sample 3: 95.7633


In [57]:
tb2 = {'Sample': ['Sample 1', 'Sample 2', 'Sample 3'],
        'Mean': [td_mean_s1,td_mean_s2,td_mean_s3],
        'Standard Deviation': [td_std_s1,td_std_s2,td_std_s3],
        'Standard Error': [td_se_s1,td_se_s2,td_se_s3]}

total_discount_summeries = pd.DataFrame(tb2)

# Print the DataFrame
print(total_discount_summeries)

     Sample         Mean  Standard Deviation  Standard Error
0  Sample 1  1306.926691         1822.416638      112.161991
1  Sample 2  1631.332640         2478.036861      128.827002
2  Sample 3  1421.948903         2386.408750       95.763284


In [29]:
data1 = {'Confidense Interval':['90%','95%','99%']
        , 'Sample Size':[sample_size_ci90,sample_size_ci95,sample_size_ci99]
        , 'Discount Mean':[td_mean_s1,td_mean_s2,td_mean_s3]
        ,'Real Discount Mean':[population_total_discount_received_mean,population_total_discount_received_mean,population_total_discount_received_mean]}


In [59]:
df1=pd.DataFrame(data1)
df1


,Confidense Interval,Sample Size,Discount Mean,Real Discount Mean
0,90%,264,1306.926691,1459.16
1,95%,370,1631.332640,1459.16
2,99%,621,1421.948903,1459.16


In [58]:
df1['Absolute Error'] = abs(df1['Real Discount Mean'] - df1['Discount Mean'])
df1

,Confidense Interval,Sample Size,Discount Mean,Real Discount Mean,Absolute Error
0,90%,264,1306.926691,1459.16,152.233309
1,95%,370,1631.332640,1459.16,172.172640
2,99%,621,1421.948903,1459.16,37.211097
